In [18]:
import numpy as np
import numpy.ma as ma
import pandas as pd 
import matplotlib as mpl
import seaborn as sb


Pulsar = pd.read_csv("pulsar_stars.csv")
df = pd.DataFrame(Pulsar)
print (df)
df.columns.str.strip()


        Mean of the integrated profile  \
0                           140.562500   
1                           102.507812   
2                           103.015625   
3                           136.750000   
4                            88.726562   
...                                ...   
17893                       136.429688   
17894                       122.554688   
17895                       119.335938   
17896                       114.507812   
17897                        57.062500   

        Standard deviation of the integrated profile  \
0                                          55.683782   
1                                          58.882430   
2                                          39.341649   
3                                          57.178449   
4                                          40.672225   
...                                              ...   
17893                                      59.847421   
17894                                      49.4

Index(['Mean of the integrated profile',
       'Standard deviation of the integrated profile',
       'Excess kurtosis of the integrated profile',
       'Skewness of the integrated profile', 'Mean of the DM-SNR curve',
       'Standard deviation of the DM-SNR curve',
       'Excess kurtosis of the DM-SNR curve', 'Skewness of the DM-SNR curve',
       'target_class'],
      dtype='object')